In [5]:
def findPrime(x):
    prime=[2, 3] #
    notPrime=[]
    numbers=[2,3]
    exclude=4 #done to exclude 0 and 1 as they are unique numbers, exclude 2 and 3 as to get the list started.
    for i in range(x-exclude+1):
        num=i+exclude
        status=True
        
        for j in prime:
            if num%j==0:
                status=False
        numbers.append(num)
        if status:
            prime.append(num)
        else:
            notPrime.append(num)
    return(prime, notPrime, numbers)

number=10000
prime, notPrime,numbers=findPrime(number)
#print(prime, notPrime)

        

In [6]:
import numpy as np

label=np.zeros((number-1,1))
print(label)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [7]:
for i in prime:
    label[i-2]=1
#print(label)

from keras.utils import np_utils
label_final = np_utils.to_categorical(np.array(label), 2)
print(label)
print(label_final)

[[1.]
 [1.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [8]:
data=np.zeros((number-1,1))
for i in range(number-1):
    data[i]=i+2
#print(data)
data_final = np_utils.to_categorical(np.array(data), number+1)
print(data_final)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_final, label_final, test_size=0.2, random_state=42)

In [15]:
def num(x, number):
    zeros=np.zeros(number+1)
    zeros[x]=1
    #print(zeros)
    return(zeros)

def predPrime(x):
    #Prime_model.predict(np.array([num(x, number)]))
    #print(Prime_model.predict(np.array([num(x, number)]))[0])
    return(Prime_model.predict(np.array([num(x, number)]))[0][0],Prime_model.predict(np.array([num(x, number)]))[0][1])

In [17]:
#neural network(s):
#full:
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import math

from keras.callbacks import ModelCheckpoint, EarlyStopping 

print(X_train.shape[1])
for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                for m in range(2):
                    Prime_model = Sequential()
                    Prime_model.add(Dense(2**i, input_shape=(X_train.shape[1],)))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**j, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**k, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**l, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**m, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2, activation='softmax'))
                    Prime_model.summary()



                    #compile network:
                    Prime_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


                    #running network:
                    
                    ### TODO: specify the number of epochs that you would like to use to train the model.

                    epochs = 100

                    ### Do NOT modify the code below this line.

                    checkpointer = ModelCheckpoint(filepath='G:\Machine Learning\Prime-Numbers-master\Dense\weights.bestdensei'+str(i)+'j'+str(j)+'k'+str(k)+'l'+str(l)+'m'+str(m)+'.from_scratch.hdf5', 
                                                   verbose=1, save_best_only=True)
                    checkpointer2=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

                    Prime_model.fit(X_train, y_train, 
                                    validation_data=(X_test, y_test),
                                    epochs=epochs, batch_size=7, callbacks=[checkpointer, checkpointer2], verbose=1)
                    
                    #load model
                    Prime_model.load_weights('G:\Machine Learning\Prime-Numbers-master\Dense\weights.bestdensei'+str(i)+'j'+str(j)+'k'+str(k)+'l'+str(l)+'m'+str(m)+'.from_scratch.hdf5')
                    
                    for n in prime:
                        probNotPrime, probPrime= predPrime(n)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                    
                    for o in notPrime:
                        probNotPrime, probPrime= predPrime(o)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                        
                    #data1D=[]
                    notPrimeArray=[]
                    
                    for p in numbers:
                        probNotPrime, probPrime=predPrime(p)
                        notPrimeArray.append(probNotPrime)
                        
                        
                    labelArray=[]
                    for q in label:
                        labelArray.append(q[0])

                    notPrimeArray2D=[] 
                    for r in notPrimeArray:
                        notPrimeArray2D.append([r])
    
    
                    X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(notPrimeArray2D, 
                                                                                                labelArray, test_size=0.2, random_state=42)

                    clf=AdaBoostClassifier(n_estimators=50)

                    print (len(X_train_boost), len(y_train_boost))

                    #print (X_train_boost)
                    #print (y_train_boost)

                    clf.fit(X_train_boost, y_train_boost)
                    
                    def boostpredict(x):
                        return(clf.predict(x))
                    
                    accuracyPrime=0
                    for s in prime:
                        probNotPrime, probPrime= predPrime(s)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                        result=boostpredict(probNotPrime)
                        #print('i=', i, '  probNotPrime=', result)
                        if result==[1.]:
                            accuracyPrime+=1
                            
                    accuracyNotPrime=0
                    for t in notPrime:
                        probNotPrime, probPrime= predPrime(t)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                        result=boostpredict(probNotPrime)
                        #print('i=', i, '  probNotPrime=', result)
                        if result==[0.]:
                            accuracyNotPrime+=1

                            
                            
                    print('i=', i, ', j=', j, ', k=', k, ', l=', l, ', m=', m,
                          ', prime accuracy=', 100.0*accuracyPrime/len(prime),
                          ', not prime accuracy=', 100.0*accuracyNotPrime/len(notPrime))

                    

10001
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 1)                 10002     
_________________________________________________________________
dropout_31 (Dropout)         (None, 1)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 1)                 2         
_________________________________________________________________
dropout_32 (Dropout)         (None, 1)                 0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 2         
_________________________________________________________________
dropout_33 (Dropout)         (None, 1)                 0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 2         
____

KeyboardInterrupt: 

In [8]:
#NN:
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import math

from keras.callbacks import ModelCheckpoint, EarlyStopping 

print(X_train.shape[1])
for i in range(13):
    for j in range(13):
        for k in range(13):
            for l in range(13):
                for m in range(13):
                    Prime_model = Sequential()
                    Prime_model.add(Dense(2**i, input_shape=(X_train.shape[1],)))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**j, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**k, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**l, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2**m, activation='relu'))
                    Prime_model.add(Dropout(.3))
                    Prime_model.add(Dense(2, activation='softmax'))
                    Prime_model.summary()



                    #compile network:
                    Prime_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


                    #running network:
                    
                    ### TODO: specify the number of epochs that you would like to use to train the model.

                    epochs = 100

                    ### Do NOT modify the code below this line.

                    checkpointer = ModelCheckpoint(filepath='G:\Machine Learning\Prime-Numbers-master\Dense\weights.bestdensei'+str(i)+'j'+str(j)+'k'+str(k)+'l'+str(l)+'m'+str(m)+'.from_scratch.hdf5', 
                                                   verbose=1, save_best_only=True)
                    checkpointer2=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

                    Prime_model.fit(X_train, y_train, 
                                    validation_data=(X_test, y_test),
                                    epochs=epochs, batch_size=7, callbacks=[checkpointer, checkpointer2], verbose=1)
                    
                    

                    

In [ ]:
#SL(s):
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, AveragePooling1D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import math

from keras.callbacks import ModelCheckpoint, EarlyStopping 

print(X_train.shape[1])
for i in range(13):
    for j in range(13):
        for k in range(13):
            for l in range(13):
                for m in range(13):
                                        
                    #load model
                    Prime_model.load_weights('G:\Machine Learning\Prime-Numbers-master\Dense\weights.bestdensei'+str(i)+'j'+str(j)+'k'+str(k)+'l'+str(l)+'m'+str(m)+'.from_scratch.hdf5')
                    
                    for n in prime:
                        probNotPrime, probPrime= predPrime(n)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                    
                    for o in notPrime:
                        probNotPrime, probPrime= predPrime(o)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                        
                    #data1D=[]
                    notPrimeArray=[]
                    
                    for p in numbers:
                        probNotPrime, probPrime=predPrime(p)
                        notPrimeArray.append(probNotPrime)
                        
                        
                    labelArray=[]
                    for q in label:
                        labelArray.append(q[0])

                    notPrimeArray2D=[] 
                    for r in notPrimeArray:
                        notPrimeArray2D.append([r])
    
    
                    X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(notPrimeArray2D, 
                                                                                                labelArray, test_size=0.2, random_state=42)

                    clf=AdaBoostClassifier(n_estimators=50)

                    print (len(X_train_boost), len(y_train_boost))

                    #print (X_train_boost)
                    #print (y_train_boost)

                    clf.fit(X_train_boost, y_train_boost)
                    
                    def boostpredict(x):
                        return(clf.predict(x))
                    
                    accuracyPrime=0
                    for s in prime:
                        probNotPrime, probPrime= predPrime(s)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                        result=boostpredict(probNotPrime)
                        #print('i=', i, '  probNotPrime=', result)
                        if result==[1.]:
                            accuracyPrime+=1
                            
                    accuracyNotPrime=0
                    for t in notPrime:
                        probNotPrime, probPrime= predPrime(t)
                        #print('i=', i, '  probNotPrime=', probNotPrime, '  probPrime=', probPrime)
                        result=boostpredict(probNotPrime)
                        #print('i=', i, '  probNotPrime=', result)
                        if result==[0.]:
                            accuracyNotPrime+=1

                            
                            
                    print('i=', i, ', j=', j, ', k=', k, ', l=', l, ', m=', m,
                          ', prime accuracy=', 100.0*accuracyPrime/len(prime),
                          ', not prime accuracy=', 100.0*accuracyNotPrime/len(notPrime))

                    